In [1]:
import socket
import threading
import json
import time
import uuid

# Client Configuration
BROADCAST_PORT = 59073
TCP_PORT = 59074
UUID = str(uuid.uuid4())
HEARTBEAT_TIMEOUT = 10

server_info = None


def debug(message):
    current_time = time.strftime('%H:%M:%S', time.localtime())
    print(f"[{current_time}] {message}")


def encode_message(message):
    return json.dumps(message).encode('utf-8')


def decode_message(message):
    return json.loads(message.decode('utf-8'))


class BroadcastListener(threading.Thread):
    def _init_(self):
        threading.Thread._init_(self)

    def run(self):
        global server_info
        udp_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        udp_socket.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
        udp_socket.bind(("", BROADCAST_PORT))
        debug("Listening for server broadcasts...")

        last_heartbeat = time.time()

        while True:
            data, addr = udp_socket.recvfrom(1024)
            message = decode_message(data)
            if message["cmd"] == "SERVER_BROADCAST":
                debug(f"Discovered server: {message}")
                server_info = {"ip": message["ip"], "tcp_port": message["tcp_port"]}
                last_heartbeat = time.time()

            # Detect server failure
            if time.time() - last_heartbeat > HEARTBEAT_TIMEOUT:
                debug("Server heartbeat not detected. Possible failure.")
                break


class TCPClient(threading.Thread):
    def _init_(self):
        threading.Thread._init_(self)

    def run(self):
        global server_info
        while not server_info:
            time.sleep(1)

        tcp_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        tcp_socket.connect((server_info["ip"], server_info["tcp_port"]))
        debug(f"Connected to server at {server_info['ip']}:{server_info['tcp_port']}")

        # Register client
        message = {
            "cmd": "REGISTER",
            "uuid": UUID,
            "ip": socket.gethostbyname(socket.gethostname()),
            "tcp_port": TCP_PORT
        }
        tcp_socket.send(encode_message(message))
        response = tcp_socket.recv(1024)
        debug(f"Server response: {decode_message(response)}")

        # Send messages
        for i in range(5):
            time.sleep(1)
            message = {"cmd": "MESSAGE", "content": f"Message {i+1} from {UUID}"}
            tcp_socket.send(encode_message(message))
            response = tcp_socket.recv(1024)
            debug(f"Server response: {decode_message(response)}")

        tcp_socket.close()


if __name__ == "__main__":
    try:
        debug("Starting client...")
        broadcast_listener = BroadcastListener()
        broadcast_listener.start()

        tcp_client = TCPClient()
        tcp_client.start()
    except KeyboardInterrupt:
        debug("Client shutting down...")

[16:00:16] Starting client...
[16:00:16] Listening for server broadcasts...
[16:00:19] Discovered server: {'cmd': 'SERVER_BROADCAST', 'ip': '192.168.233.79', 'tcp_port': 59074, 'timestamp': 1737644420.0951629}
[16:00:20] Connected to server at 192.168.233.79:59074
[16:00:20] Server response: {'status': 'OK'}
[16:00:21] Server response: {'status': 'OK'}
[16:00:22] Server response: {'status': 'OK'}
[16:00:23] Server response: {'status': 'OK'}
[16:00:24] Server response: {'status': 'OK'}
[16:00:24] Discovered server: {'cmd': 'SERVER_BROADCAST', 'ip': '192.168.233.79', 'tcp_port': 59074, 'timestamp': 1737644425.096638}
[16:00:25] Server response: {'status': 'OK'}
[16:00:29] Discovered server: {'cmd': 'SERVER_BROADCAST', 'ip': '192.168.233.79', 'tcp_port': 59074, 'timestamp': 1737644430.0986612}
[16:00:34] Discovered server: {'cmd': 'SERVER_BROADCAST', 'ip': '192.168.233.79', 'tcp_port': 59074, 'timestamp': 1737644435.1001174}
[16:00:39] Discovered server: {'cmd': 'SERVER_BROADCAST', 'ip': 